In [ ]:
# mostly torch imports and plot imports
import os
import sys
import cv2
# mostly torch imports and plot imports
import torch
import shutil
import glob
import pickle
import random
random.seed(42)
# import colorama
# from colorama import Fore, Style
import numpy as np
np.random.seed(42)
import torch.utils
import torchvision
from torch import optim
import torch.distributions
torch.manual_seed(42)
from tqdm import tqdm
import torch.nn as nn
from torchsummary import summary
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from matplotlib import rc, rcParams
from numpy import sin

use_cuda = torch.cuda.is_available()
print('use_cuda: {}'.format(use_cuda))
device = torch.device("cuda" if use_cuda else "cpu")
print("Device to be used : ",device)

use_cuda: True
Device to be used :  cuda


In [ ]:

class ClassificationNetwork(nn.Module):
    def __init__(self):
        super(ClassificationNetwork, self).__init__()
        self.fc = nn.Linear(10, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

# Create an instance of the network
model = ClassificationNetwork()

In [ ]:
class DataGenerator(Dataset):
    def __init__(self, X_VALUES, Y_VALUES, choice):
        self.X = X_VALUES
        self.Y = Y_VALUES
        self.choice = choice

    def __get__number__(self, str_num):
        return np.fromiter((int(bit) for bit in str_num), dtype=np.int8)

    # NUMBER OF FILES IN THE DATASET
    def __len__(self):
        return len(self.X)

    # GETTING SINGLE PAIR OF DATA
    def __getitem__(self, idx):
        # print("X = ",self.X)
        # print("idx = ",idx)
        X_numpy = self.__get__number__(self.X[idx])
        X_numpy_reshaped = X_numpy.reshape((1, 10))
        # print("X_numpy_reshaped = ", torch.FloatTensor(X_numpy_reshaped))
        # print("Y = ", torch.FloatTensor(self.Y[idx]))
        return torch.FloatTensor(X_numpy_reshaped), torch.FloatTensor(self.Y[idx])


In [ ]:
def load_data(X_VALUES, Y_VALUES, choice="train", batch_size=2, num_workers=10, shuffle=True):
    dataset = DataGenerator(X_VALUES, Y_VALUES, choice=choice)
    data_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)
    return data_loader


# save checkpoint in pytorch
def save_ckp(checkpoint, checkpoint_path):
    torch.save(checkpoint, checkpoint_path)


# load checkpoint in pytorch
def load_ckp(checkpoint_path, model, model_opt):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    model_opt.load_state_dict(checkpoint['optimizer'])
    return model, model_opt, checkpoint['epoch']

In [ ]:
def train_epoch(train_loader, model, optimizer, epoch):
    print("\n\n---------------------------------------------------------------------------------------------------------------\n")

    progress_bar = tqdm(enumerate(train_loader))
    total_loss = 0.0

    N = 0
    for step, (x, y) in progress_bar:
        # if mask_1 is None and low_res_img is None:
        #     continue
        model.train()

        #TRANSFERRING DATA TO DEVICE
        x = x.to(device)
        y = y.to(device)

        # clear the gradient
        optimizer.zero_grad()

        #GETTING THE PREDICTED IMAGE
        print("x shape == ",x.shape)
        pred_y = model.forward(x)

        #LOSS FUNCTIONS
        BCELOSS = nn.BCELoss()

        #CALCULATING LOSSES
        BCE_loss = BCELOSS(pred_y, y)

        #LOSS TAKEN INTO CONSIDERATION
        loss = BCE_loss


        # CALCULATING METRICS
        total_loss += loss

        # print(loss)

        #BACKPROPAGATING THE LOSS
        loss.backward()
        optimizer.step()

        #DISPLAYING THE LOSS
        progress_bar.set_description("Epoch: {} -  Loss: {} ".format(epoch, loss))


    with open("history/train_logs.txt", "a") as text_file:
        text_file.write("{} {}\n".format(epoch, total_loss))

    # print("Training Epoch: {} |  Total Loss: {} | Total Dice: {} | Total Jaccard: {} | N: {}".format(epoch,total_loss, total_dice, total_jacard,N))
    print("Training Epoch: {} |  Loss: {}".format(epoch, total_loss))

    return model, optimizer



def test_epoch(test_loader, model, optimizer, epoch):

    progress_bar = tqdm(enumerate(test_loader))
    total_loss = 0.0

    #SETTING THE NUMBER OF IMAGES TO CHECK AFTER EACH ITERATION
    no_img_to_write = 20

    total_loss = 0.0
    for step, (x, y) in progress_bar:

        x = x.to(device)
        y = y.to(device)


        #PREDICTED IMAGE
        pred_y = model.forward(x)

        #LOSS FUNCTIONS
        BCELOSS = nn.BCELoss()

        model.eval()

        #CALCULATING LOSSES
        BCE_loss = BCELOSS(pred_y, y)

        #LOSS TAKEN INTO CONSIDERATION
        loss = BCE_loss

        # CALCULATING METRICS
        total_loss += loss


        progress_bar.set_description("Epoch: {} -  Loss: {} ".format(epoch, total_loss))


    with open("history/test_logs.txt", "a") as text_file:
        text_file.write("{} {}\n".format(epoch, total_loss))

    print("Test Epoch: {} |  Loss: {}".format(epoch, total_loss))
    print("---------------------------------------------------------------------------------------------------------------")



In [ ]:
def train_val_test(train_loader, test_loader, model, optimizer, n_epoch, resume):

    #PATH TO SAVE THE CHECKPOINT
    checkpoint_path = "checkpoint/classification_net.pt"

    epoch = 0
    #IF TRAINING IS TO RESUMED FROM A CERTAIN CHECKPOINT
    if resume:
        model, optimizer, epoch = load_ckp(
            checkpoint_path, model, optimizer)

    while epoch <= n_epoch:
        epoch += 1
        model, optimizer = train_epoch(train_loader, model, optimizer, epoch)

        #CHECKPOINT CREATION
        checkpoint = {'epoch': epoch+1, 'state_dict': model.state_dict(),
                      'optimizer': optimizer.state_dict()}

        #CHECKPOINT SAVING
        save_ckp(checkpoint, checkpoint_path)
        print("Checkpoint Saved")

    print("************************ Final Test Epoch *****************************")

    with torch.no_grad():
        test_epoch(test_loader, model, optimizer, epoch)

In [ ]:
def is_pal(X):
    return X == X[::-1]

def generate_10_bit_binary_numbers():
    binary_numbers = []

    for i in range(2**10):
        binary_str = format(i, '010b')
        binary_numbers.append(binary_str)

    return binary_numbers


# Generate and store all 10-bit binary numbers
binary_numbers_list = generate_10_bit_binary_numbers()


Y_VALUES = []
X_VALUES = []

# Print and/or use the generated binary numbers as needed
for binary_number in binary_numbers_list:
    #print(binary_number)
    if is_pal(binary_number) == True:
        X_VALUES.append(binary_number)
        Y_VALUES.append(1)
    else:
        X_VALUES.append(binary_number)
        Y_VALUES.append(0)

In [ ]:

def main():

    train_till = int(0.8*len(X_VALUES))

    X_VALUES_TRAIN = X_VALUES[:train_till]
    Y_VALUES_TRAIN = Y_VALUES[:train_till]

    X_VALUES_TEST = X_VALUES[train_till:]
    Y_VALUES_TEST = Y_VALUES[train_till:]

    print("Total Number of Training data : ", len(X_VALUES_TRAIN))
    print("Total Number of Testing data : ", len(X_VALUES_TEST))


    # CREATING THE TRAIN LOADER
    train_loader = load_data(
        X_VALUES_TRAIN, Y_VALUES_TRAIN, choice="train", batch_size=16, num_workers=1, shuffle=True)

    # #CREATING THE TEST LOADER
    test_loader = load_data(
        X_VALUES_TEST, Y_VALUES_TEST, choice="test", batch_size=1, num_workers=1, shuffle=False)

    #CALLING THE MODEL
    model = ClassificationNetwork()
    model = model.to(device)

    summary(model, input_size=(1, 10))

    #DEFINING THE OPTIMIZER
    optimizer = optim.Adam(
        [p for p in model.parameters() if p.requires_grad], lr=1e-05, weight_decay=5e-4)

    n_epoch = 200

    #INDICATOR VARIABLE TO RESUME TRAINING OR START AFRESH
    resume = False
    train_val_test(train_loader, test_loader, model, optimizer, n_epoch, resume)





In [ ]:
if __name__ == "__main__":
    print("--- Starting the main function ----")
    main()

--- Starting the main function ----
Total Number of Training data :  819
Total Number of Testing data :  205
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 1]              11
           Sigmoid-2                 [-1, 1, 1]               0
Total params: 11
Trainable params: 11
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


---------------------------------------------------------------------------------------------------------------



0it [00:00, ?it/s]

x shape ==  torch.Size([16, 1, 10])


ValueError: Using a target size (torch.Size([16, 0])) that is different to the input size (torch.Size([16, 1, 1])) is deprecated. Please ensure they have the same size.

In [ ]:
X = torch.tensor([0., 0., 1., 0., 0., 1., 1., 1., 0., 1.])
print(X.shape)

torch.Size([10])
